In [5]:
import logging
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
import os
from os.path import join
import sys
import pickle
import csv
from transformers import BertTokenizer,BertForSequenceClassification
from transformers.optimization import AdamW

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(6)

In [7]:
# set dimension of interest
dim = 'romance'

In [8]:
DATA_DIR = 'data/%s'%dim # directory of where the train/test/dev files are stored
OUTPUT_DIR = 'weights/BERT/%s' %dim # where the model weights will be stored
BERT_MODEL = 'bert-base-cased' # BERT model type
CACHE_DIR = 'cache/' # where BERT will look for pre-trained models to load parameters from

num_labels = 2

OUTPUT_MODE = 'classification'
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [9]:
# load model
model = BertForSequenceClassification.from_pretrained(BERT_MODEL,cache_dir=CACHE_DIR, num_labels=num_labels)

C:\Users\fonta\anaconda3\envs\cuda_20231105\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fonta\PycharmProjects\10_dimensions_classifier\cache. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# load pretrained BERT model for specific dimension
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
model.load_state_dict(torch.load(output_model_file))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [11]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)

In [12]:
## enter a sentence
rand_sentence = '### insert sentence here ###'
rand_sentence = 'this man is crazy in love'

In [13]:
input_ids = torch.tensor([tokenizer.encode(rand_sentence,add_special_tokens=True)]).cuda()
with torch.no_grad():
    outputs = model(input_ids)[0]
    score = torch.softmax(outputs,1)
print("%s score for sentence: %1.3f"%(dim,score[0,1].item()))

romance score for sentence: 0.981
